# Download data

In [0]:
from getpass import getpass
import os
user = 'ahmadelsallab'
key = '6b7ffe97ff5bc0656e325b746b72fa31'

if '.kaggle' not in os.listdir('/root'):
    !mkdir ~/.kaggle
!touch /root/.kaggle/kaggle.json
!chmod 666 /root/.kaggle/kaggle.json
with open('/root/.kaggle/kaggle.json', 'w') as f:
    f.write('{"username":"%s","key":"%s"}' % (user, key))
!chmod 600 /root/.kaggle/kaggle.json


!kaggle competitions download -c jigsaw-unintended-bias-in-toxicity-classification
!chmod 777 *
!ls

sample_submission.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
test.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
train.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
sample_data	       sample_submission.csv.zip  test.csv.zip	train.csv.zip
sample_submission.csv  test.csv			  train.csv


In [0]:
!unzip sample_submission.csv.zip
!unzip test.csv.zip
!unzip train.csv.zip
!ls

Archive:  sample_submission.csv.zip
replace sample_submission.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: N
Archive:  test.csv.zip
replace test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: N
Archive:  train.csv.zip
replace train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: N
sample_data	       sample_submission.csv.zip  test.csv.zip	train.csv.zip
sample_submission.csv  test.csv			  train.csv


# Explore data samples

In [0]:
import pandas as pd
train_df = pd.read_csv('train.csv')
train_df.head()

,id,target,comment_text,severe_toxicity,obscene,identity_attack,insult,threat,asian,atheist,bisexual,black,buddhist,christian,female,heterosexual,hindu,homosexual_gay_or_lesbian,intellectual_or_learning_disability,jewish,latino,male,muslim,other_disability,other_gender,other_race_or_ethnicity,other_religion,other_sexual_orientation,physical_disability,psychiatric_or_mental_illness,transgender,white,created_date,publication_id,parent_id,article_id,rating,funny,wow,sad,likes,disagree,sexual_explicit,identity_annotator_count,toxicity_annotator_count
0,59848,0.000000,"This is so cool. It's like, 'would you want yo...",0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-09-29 10:50:41.987077+00,2,NaN,2006,rejected,0,0,0,0,0,0.0,0,4
1,59849,0.000000,Thank you!! This would make my life a lot less...,0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-09-29 10:50:42.870083+00,2,NaN,2006,rejected,0,0,0,0,0,0.0,0,4
2,59852,0.000000,This is such an urgent design problem; kudos t...,0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-09-29 10:50:45.222647+00,2,NaN,2006,rejected,0,0,0,0,0,0.0,0,4
3,59855,0.000000,Is this something I'll be able to install on m...,0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-09-29 10:50:47.601894+00,2,NaN,2006,rejected,0,0,0,0,0,0.0,0,4
4,59856,0.893617,haha you guys are a bunch of losers.,0.021277,0.0,0.021277,0.87234,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2015-09-29 10:50:48.488476+00,2,NaN,2006,rejected,0,0,0,1,0,0.0,4,47


In [0]:
import pandas as pd
test_df = pd.read_csv('test.csv')
test_df.head()

,id,comment_text
0,7000000,Jeff Sessions is another one of Trump's Orwell...
1,7000001,I actually inspected the infrastructure on Gra...
2,7000002,No it won't . That's just wishful thinking on ...
3,7000003,Instead of wringing our hands and nibbling the...
4,7000004,how many of you commenters have garbage piled ...


# Data pre-processing

In [0]:
import numpy as np

In [0]:
trn_texts = list(train_df['comment_text'])
tst_texts = list(test_df['comment_text'])
texts = trn_texts + tst_texts

In [0]:
from keras.preprocessing.text import Tokenizer, text_to_word_sequence

tokenizer = Tokenizer()


Using TensorFlow backend.


In [0]:
tokenizer.fit_on_texts(texts)


In [0]:
del texts
del tst_texts
del test_df

In [0]:
binary_trn_texts = tokenizer.texts_to_sequences(trn_texts)

In [0]:

# TODO: Calc max seqlen
maxlen = max([len(s) for s in binary_trn_texts])
maxlen
#idx = np.argmax([len(s) for s in binary_trn_texts])
#trn_texts[idx]

317

In [0]:
from keras.preprocessing.sequence import pad_sequences

x_train = np.array(pad_sequences(binary_trn_texts, maxlen=maxlen))


In [0]:
del binary_trn_texts

In [0]:
x_train.shape

(1804874, 317)

In [0]:
y_train = train_df['target']
y_train.shape

(1804874,)

In [0]:

# Shuffle
'''
np.random.seed(42)
trn_idx = np.random.permutation(len(trn_texts))

x_train = x_train[trn_idx]
y_train = y_train[trn_idx]
'''


In [0]:
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.1, random_state=42, shuffle=True)

# Model

In [0]:
model_name = 'cnn_lstm_text_only'

In [0]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Embedding
from keras.layers import LSTM
from keras.layers import Conv1D, MaxPooling1D

vocab_size = len(tokenizer.word_index)
embedding_size = 300


# Training
batch_size = 256
epochs = 20

# Model 1: Simple LSTM, with Embedding mask_zero
model = Sequential()
model.add(Embedding(vocab_size+1, embedding_size, mask_zero=True, input_length=maxlen))
model.add(LSTM(lstm_output_size))
model.add(Dropout(0.25))
model.add(Dense(500))
model.add(Dropout(0.25))
model.add(Dense(200))
model.add(Dropout(0.25))
model.add(Dense(100))
model.add(Dropout(0.25))
model.add(Dense(1, activation='sigmoid'))


'''
# Model 2: BiGRU
sentence_input = Input(shape=(maxlen,), dtype='int32')
embedded_sequences = Embedding(vocab_size+1, embedding_size, mask_zero=True)(sentence_input)
l_lstm = Bidirectional(GRU(300))(embedded_sequences)
l_dense = Dense(100, activation='relu')(l_lstm)
output = Dense(1, activation='sigmoid')(l_dense)
model = Model(sentence_input, output)
'''

'''
# Model 3: BiGRU + distributed dense
sentence_input = Input(shape=(maxlen,), dtype='int32')
embedded_sequences = Embedding(vocab_size+1, embedding_size, mask_zero=True)(sentence_input)
l_lstm = Bidirectional(GRU(1000, return_sequences=True))(embedded_sequences)
l_word_enc = TimeDistributed(Dense(500))(l_lstm)
l_lstm_2 = LSTM(500)(l_word_enc)
l_dense = Dense(200, activation='relu')(l_word_enc)
l_dense = Dense(100, activation='relu')(l_dense)
output = Dense(1, activation='sigmoid')(l_lstm_2)
model = Model(sentence_input, output)
'''



model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 317, 100)          41004600  
_________________________________________________________________
dropout_3 (Dropout)          (None, 317, 100)          0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 315, 256)          77056     
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 105, 256)          0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 1000)              5028000   
_________________________________________________________________
dense_5 (Dense)              (None, 500)               500500    
_________________________________________________________________
dense_6 (Dense)              (None, 200)               100200    
__________

In [0]:
# Mount GDrive
from google.colab import drive
drive.mount('/content/gdrive')
gdrive_path = 'gdrive/My Drive/Colab Notebooks/JIGSAW/dat'

from keras.callbacks import ModelCheckpoint, Callback
filepath = os.path.join(gdrive_path, 'jigsaw_' + model_name + '.h5')
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')




Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
from keras.callbacks import Callback
from sklearn.metrics import roc_auc_score, auc
class roc_callback(Callback):
    def __init__(self, x_val, y_val):
        
        self.x_val = x_val
        # y_val is continous, but according to the competetion, the thresh is at 0.5, so we can convert to binary
        # The *1 is to convert False/True into 0/1    
        self.y_val = np.array((y_val>0.5)*1)
        
    
    def on_train_begin(self, logs={}):
        return
 
    def on_train_end(self, logs={}):
        return
 
    def on_epoch_begin(self, epoch, logs={}):
        return
 
    def on_epoch_end(self, epoch, logs={}):        
        y_pred = self.model.predict(self.x_val)

        val_auc  = roc_auc_score(self.y_val, model.predict(x_val))

        
        print('Epoch: ' + str(epoch) + ' AUC: ' + str(val_auc))
        
        return
 
    def on_batch_begin(self, batch, logs={}):
        return
 
    def on_batch_end(self, batch, logs={}):
        return 

In [0]:
model.compile(loss='mse',
              optimizer='adam',
              metrics=['accuracy'])

callbacks_lst = [checkpoint, roc_callback(x_val, y_val)]

print('Train...')
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,          
          validation_data=(x_val, y_val),
          #validation_split=0.2,
          callbacks=callbacks_lst)

Train...
Train on 1624386 samples, validate on 180488 samples
Epoch 1/20


In [0]:
from sklearn.metrics import roc_auc_score, auc
fpr, tpr, thresh  = roc_auc_score(y_val, model.predict(x_val))
val_auc = auc(fpr, tpr)
val_auc

# Prepare test data

In [0]:
import pandas as pd
test_df = pd.read_csv('test.csv')
test_df.head()
tst_texts = list(test_df['comment_text'])
binary_tst_texts = tokenizer.texts_to_sequences(test_texts)
x_test = np.array(pad_sequences(binary_tst_texts, maxlen=maxlen))

In [0]:
y_test = model.predict(x_test)

sub_df = pd.DataFrame({'id': test_df['id'], 'prediction':y_test})

sub_df.to_csv('submission_' + model_name + '.csv')
                      
